In [11]:
import time
import requests
import pandas as pd
import plotly.express as px

from enum import Enum
from decimal import Decimal
from datetime import datetime
from pydantic import ValidationError
from pydantic.dataclasses import dataclass

In [12]:
class Timeframe(Enum):
    HOURLY = "1h"
    DAILY = "1d"
    WEEKLY = "1w"
    MONTHLY = "1M"

@dataclass
class Candle:
    time: datetime
    open: Decimal
    high: Decimal
    low: Decimal
    close: Decimal
    volume: Decimal

def exluded_symbols(market: str, symbols: list[str]) -> bool:
    """Checks if a excluded symbol is part of a given market and returns True if it is."""
    if len(symbols) > 0:
        for item in symbols:
            if market.startswith(item) or market.endswith(item):
                return True
    return False

class Binance():
    BASEURL = "https://api.binance.com"

    def __init__(self) -> None:
        pass

    def markets(self, market: str, exclude: list[str] = []) -> list[str]:
        url = Binance.BASEURL + "/api/v3/ticker/price"
        r = requests.get(url)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        data = filter(lambda x: x.endswith(market) and not exluded_symbols(x, exclude), map(lambda x: x.get("symbol", ""), r.json()))
        return list(data)

    def kline(self, symbol: str, interval: Timeframe) -> list[Candle]:
        url = Binance.BASEURL + "/api/v3/klines"
        payload = { "symbol": symbol, "interval": interval.value }
        r = requests.get(url, params=payload)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        klines = []
        for kline in r.json():
            candle = Candle(*kline[:6])
            klines.append(candle)

        return klines

In [13]:
stablecoins = ["TUSD", "BUSD", "USDC", "PAX", "USDP", "DAI", "GUSD", "USDD", "USTC", "UST", "USDS"]

binance = Binance()
markets = binance.markets("USDT", stablecoins)

print(f"Downloading {len(markets)} markets...")
market_data = {}
for i in range(len(markets)):
    if i % 20 == 0:
        time.sleep(5)
    market = markets[i]
    market_data[market] = binance.kline(market, Timeframe.DAILY)
print("Finished downloading.")

Finished downloading.


In [14]:
btcusdt = pd.DataFrame.from_dict(market_data["BTCUSDT"]).set_index("time")

In [15]:
btcusdt.head()

,open,high,low,close,volume
time,,,,,
2022-01-12 00:00:00+00:00,42729.29000000,44322.00000000,42450.00000000,43902.66000000,33943.29280000
2022-01-13 00:00:00+00:00,43902.65000000,44500.00000000,42311.22000000,42560.11000000,34910.87762000
2022-01-14 00:00:00+00:00,42558.35000000,43448.78000000,41725.95000000,43059.96000000,32640.88292000
2022-01-15 00:00:00+00:00,43059.96000000,43800.00000000,42555.00000000,43084.29000000,21936.05616000
2022-01-16 00:00:00+00:00,43084.29000000,43475.00000000,42581.79000000,43071.66000000,20602.35271000
